In [33]:
# Cell 1: Imports and setup
import asyncio
import nest_asyncio
from dataclasses import replace
from langchain_core.prompts import ChatPromptTemplate
from pydantic.v1 import ValidationError
import os
from orchestrator.models.models import planner_model
from orchestrator.models.sequence_planner import SequencePlanner
from orchestrator.models.solution_agent import SolutionAgent
from orchestrator.types.plan import Plan
from orchestrator.types.plan_execute_state import PlanEntry, PlanExecuteState

nest_asyncio.apply()

# Cell 2: Define planner prompt and chain
planner_prompt = ChatPromptTemplate.from_messages(
    [
        (
            'system',
            """For the given objective, come up with a simple step by step plan. \
            This plan should involve individual tasks, that if executed correctly will yield the correct answer. Do not add any superfluous steps. \
            The result of the final step should be the final answer. Make sure that each step has all the information needed - do not skip steps.""",
        ),
        ('human', '{objective}'),
    ]
)

planner_prompt = ChatPromptTemplate.from_messages(
    [
        (
            'system',
            """For the given objective, come up with a simple step by step plan. \
This plan should involve individual tasks, that if executed correctly will yield the correct answer. Do not add any superfluous steps. \
The result of the final step should be the final answer. Make sure that each step has all the information needed - do not skip steps.""",
        ),
        ('human', '{objective}'),
    ]
)

model = planner_model.with_structured_output(schema=Plan, include_raw=False)

planner_chain = planner_prompt | model


async def plan_step(state: PlanExecuteState):
    planner = SequencePlanner(
        n_models=1, system_task=state.input, output_schema=Plan, model='gpt-4o-mini'
    )
    response = await planner.agent_runnable.ainvoke(
        {'agent_scratchpad': '', 'n_models': 1, 'system_task': state.input}
    )
    try:
        plan: Plan = Plan.validate(response)
        print(plan)
        solution_agent = SolutionAgent(
            n_models=1,
            model='gpt-4o-mini',
        )
        print(plan.message[0])
        solution = await solution_agent.ainvoke(
            {'agent_scratchpad': '', 'n_models': 1, 'task_step': plan.message[0]}
        )
        print(solution)
        plan_entry_array = [
            PlanEntry(step=step, sub_steps=None) for step in plan.message
        ]
        return replace(state, plan=plan_entry_array)
    except ValidationError as ve:
        raise Exception(ve) from ve


async def run_as_main():
    test_state = PlanExecuteState(
        input='Order a vegetarian pizza in Munich and have it delivered to Arcisstraße 21, Munich. A human will pay the delivery person upon arrival.',
        plan=None,
        past_steps=None,
        response=None,
    )
    test_return = await plan_step(test_state)
    print(test_return)

# Cell 5: Execute the main function
await run_as_main()

/Users/Lukas_1/anaconda3/envs/myenv/lib/python3.12/site-packages/langchain_core/utils/utils.py:161: UserWarning: WARNING! top_p is not default parameter.
                top_p was transferred to model_kwargs.
                Please confirm that top_p is what you intended.
  warnings.warn(


message=['Research local pizza delivery services in Munich that offer vegetarian options.', 'Select a pizza delivery service based on reviews and menu options.', "Choose a vegetarian pizza from the selected service's menu.", 'Provide the delivery address: Arcisstraße 21, Munich.', 'Place the order for the vegetarian pizza.', 'Confirm the order details and estimated delivery time.', 'Notify the human about the order and payment method upon delivery.']


/Users/Lukas_1/anaconda3/envs/myenv/lib/python3.12/site-packages/langchain_core/utils/utils.py:161: UserWarning: WARNING! top_p is not default parameter.
                top_p was transferred to model_kwargs.
                Please confirm that top_p is what you intended.
  warnings.warn(


TypeError: Object of type 'FieldInfo' is not JSON serializable

In [31]:
import json
from pydantic import BaseModel, Field, ValidationError
from typing import List

from langchain_core.prompts.prompt import PromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import Field

from orchestrator.types.base_model_config import BaseModelConfig

class Solution(BaseModel):
    result: str

class ChatOpenAI:
    def __init__(self, model, temperature, top_p):
        self.model = model
        self.temperature = temperature
        self.top_p = top_p

    def with_structured_output(self, schema, include_raw=False):
        return schema

class SolutionAgent:
    # Define Solution Agent Context
    prompt_template = """
    You are one agent in a multi-agent model of {n_models} agents.
    It is your job to provide constructive responses your team of agents can work with to solve the task at hand.
    Find the solution to the last system output: {task_step}. Your answer should be geared towards solving the task at hand, provide a "solution:" object containing the designated answer. 
    If you find sub_steps are needed to solve the matter, clearly demarcate the .json object as "steps:" and list each with bulletpoints. 

    Here is some useful information: {agent_scratchpad}
    """

    input_variables = ['agent_scratchpad', 'n_models', 'task_step']

    def __init__(
        self,
        n_models: int,
        model: str,
        temperature: int = 0,
        top_p: float = 0.1,
        agent_scratchpad: str = '',
    ):
        self.llm = ChatOpenAI(model=model, temperature=temperature, top_p=top_p)  # type: ignore
        self.n_models = n_models
        self.agent_scratchpad = agent_scratchpad
        self.prompt = PromptTemplate(
            template=self.prompt_template, input_variables=self.input_variables
        )
        self.model = self.llm.with_structured_output(schema=Solution, include_raw=False)
        self.agent_runnable = self.prompt | self.model


# Custom function to serialize objects
def custom_serializer(obj):
    if isinstance(obj, Field):
        return str(obj)
    raise TypeError(f"Object of type {obj.__class__.__name__} is not JSON serializable")

# Create a SolutionAgent instance
solution_agent = SolutionAgent(
    n_models=1,
    model='gpt-3.5-turbo',
)

# Attempt to serialize the SolutionAgent object for debugging
try:
    json_string = json.dumps(solution_agent.__dict__, default=custom_serializer)
    print(json_string)
except TypeError as e:
    print(f"Serialization error: {e}")

Serialization error: isinstance() arg 2 must be a type, a tuple of types, or a union


/Users/Lukas_1/anaconda3/envs/myenv/lib/python3.12/site-packages/langchain_core/utils/utils.py:161: UserWarning: WARNING! top_p is not default parameter.
                top_p was transferred to model_kwargs.
                Please confirm that top_p is what you intended.
  warnings.warn(


In [25]:
solution_agent.__dict__

{'llm': <__main__.ChatOpenAI at 0x10d0f8d40>,
 'n_models': 1,
 'agent_scratchpad': '',
 'prompt': PromptTemplate(input_variables=['agent_scratchpad', 'n_models', 'task_step'], template='\n    You are one agent in a multi-agent model of {n_models} agents.\n    It is your job to provide constructive responses your team of agents can work with to solve the task at hand.\n    Find the solution to the last system output: {task_step}. Your answer should be geared towards solving the task at hand, provide a "solution:" object containing the designated answer. \n    If you find sub_steps are needed to solve the matter, clearly demarcate the .json object as "steps:" and list each with bulletpoints. \n\n    Here is some useful information: {agent_scratchpad}\n    '),
 'model': __main__.Solution,
 'agent_runnable': PromptTemplate(input_variables=['agent_scratchpad', 'n_models', 'task_step'], template='\n    You are one agent in a multi-agent model of {n_models} agents.\n    It is your job to prov